# 0 摘要

GBDT集成树算法在机器学习中应用十分广泛，但在实际工程中，随着样本规模与特征规模的增大，算法性能和可扩展性遇到了瓶颈，主要原因在于**算法需要扫描所有的样本去估计所有可能分裂点的信息增益**，这一步是十分消耗时间的。

本文提出了两种创新技术分别从**样本数量**与**特征数量**上来解决这个问题：

- **Gradient-based One-side Sampling（GOSS）**：通过对小梯度样本的采样，减少计算信息增益的数据量。由于具有较大梯度样本在信息增益计算中较重要，因此采用GOSS能够在减少样本量的情况下获得准确的估计；
- **Exclusive Feature Bundling（EFB)**：通过绑定互斥特征（例如OHE）来减少特征数量。寻找最优绑定特征是一个NP-Hard问题，因此采用了贪心算法来获得近似解。

# 1 简介

传统GBDT实现会去扫描所有样本计算所有候选分裂点的信息增益，这样分裂时的时间复杂度与样本数量和特征数量成正比。因此，最直接的想法就是去减少样本与特征数量。尽管现在有一些算法通过样本的权重对数据进行采样，但由于GBDT算法本身不支持带权样本，因此这种方式无法直接应用到GBDT上。

## 1.1 Gradient-based One-side Sampling

在GBDT算法中，样本本身是无权重的。但作者发现样本不同梯度会影响信息增益的计算。具体来说，**梯度绝对值大的样本对计算信息增益的贡献更大**。因此当我们需要对样本采样的时候，首先应该保留这些梯度绝对值大的样本，而采样那些梯度绝对值较小的样本。作者证明了这种方法比随机样本采样性能更好。

> 关于梯度绝对值越大，对于信息增益计算贡献越大的理解。
> - 角度1：对一个凸函数来说，越靠近（局部）极小值的地方，函数的斜率（梯度）越接近于0，离函数的极小值越远，梯度越大。因此梯度本身可以反映当前距离最优解的远近；
> - 角度2：对于GBDT来说，采用mse损失函数时，样本在t轮的负梯度就是前t-1预测值和真实值的残差。当梯度（绝对值）越大时，意味着当前预测值和真实值之间的差距较大，当前样本没有拟合好；当梯度（绝对值）越小时，意味着当前预测值和真实值之间的差距越小，当前样本已经拟合地比较好，可以被丢掉；
> - 角度3：从信息增益角度来说，在CART中，一个子树的信息增益采用分裂后的左右叶子结点的平方误差和不分裂之前父节点的平方误差计算。因此当在计算分裂点的信息增益时，梯度（绝对值）较大的样本意味着残差也越大，因而对信息增益计算贡献就很大。

## 1.2 Exclusive Feature Bundling

在实际应用中，我们的特征往往是高维稀疏的，这就意味着有可能设计某种算法几乎无损的算法对这些特征进行降维。

在稀疏特征空间内，许多特征取值几乎都是互斥的，例如最极端的状况就是One-hot Encoding，做了One-hot Encoding后，这几列不会同时在一个样本上取非零值。类似这类特征，作者提出了通过绑定的方式减少维度。

# 2 基础知识

集成树模型最耗时的阶段在于计算信息增益，其主要包括以下步骤：
- 1.排序特征提出候选分裂点
- 2.遍历所有样本计算每个候选分裂点的信息增益
- 3.从中选出信息增益最大的特征及对应分裂点

目前比较常用的寻找分裂点的方式主要包括两种：

- 1.Pre-sort Algortithm：基于预排序好的特征值寻找最优分裂点
- 2.Histogram-based Algorithm：将连续特征值离散化为不同的bins，利用这些bins去构建特征直方图

考虑到histogram-based算法时间和空间复杂度上都更加高效，作者后续的创新都是基于这种分裂点搜索方法。

**时间复杂度计算**：

- 在histogram构建阶段，需要扫描所有样本和所有特征，因此时间复杂度是$O(\#data\times \#features)$
- 在分裂点寻找阶段，每个特征都已经被分成了不同的bin，因此时间复杂度是$O(\#bin \times \#features)$

因此作者的思路就是去减少data和features，从而优化整个计算时间。

# 3 GOSS算法

## 3.1 算法描述

GBDT的样本梯度可以作为样本的权重信息，但同时要考虑采样后的样本分布，因此GOSS分为两个步骤：

- 1.按照当前样本的梯度绝对值降序排列，选择前$a\times 100\%$个大梯度样本；
- 2.随机从剩余的样本中随机采样$b\times 100\%$个小梯度样本；
- 3.对于小梯度样本，采样后分布发生变化，此时GOSS在计算信息增益时，对小梯度样本都乘以$\frac{1-a}{b}$维持原有样本分布。

![](pictures/lightgbm-goss.png)


Histogram-based Algorithm分析：

- 最外层遍历树的层数；
- 第二层遍历当前level的结点集合。对于每个结点，获取需要使用的样本集合；
- 遍历k=1到m每个特征，开始构建直方图；可以采用最简单的分位点作为切分bin的阈值；
- 遍历样本集合中每个样本，将其第k个特征的值映射到相应的bin中，并更新直方图bin的统计量：累积y的值以及累积样本个数
- 当构建完当前第k个特征的直方图后，便可以选择第k个特征的最佳分裂点。

## 3.2 理论分析

GDBT使用决策树去学习一个从当前输入空间$\mathcal{X}^s$到梯度空间$\mathcal{G}$的函数。

假设我们有由n个i.i.d的训练样本组成的数据集合$\{x_1,\cdots,x_n\}$，其中$x_i$是一个$s$维的向量，$x_i\in \mathcal{X}^s$。定义在每一轮迭代中，损失函数关于模型输出的负梯度为$\{g_1,\cdots,g_n\}$。对于GBDT，通常使用分裂前后的方差计算信息增益。

定义$O$为在决策树中一个固定结点上的训练样本集合。按照特征$j$的值$d$分裂该结点，其相对增益量为：

$$V_{j|O}(d)=\frac{1}{n_O}(\frac{(\sum_{\{x_i\in O_{:x_{ij}\le d}\}}g_i)^2}{n_{l|O}^j(d)} + \frac{(\sum_{\{x_i\in O_{:x_{ij}\gt d}\}}g_i)^2}{n_{r|O}^j(d)})$$

其中$n_O=\sum I[x_i\in O]$，$n_{l|O}^j=\sum I[x_i\in O_{:x_{ij}\le d}]$，$n_{r|O}^j=\sum I[x_i\in O_{:x_{ij}\gt d}]$

> 通俗来讲，上面的公式就是，按照特征$j$的$d$阈值将样本分为左右两部分，使得左子树中的样本均方误差加右子树中的样本均方误差最大。
> 类比XGBoost的信息增益$\mathcal{L}_{split}=\mathcal{L}_I-(\mathcal{L}_{I_L}+\mathcal{L}_{I_R})=\frac{1}{2}\sum_{j=1}^T[\frac{(\sum_{i\in I_L}g_i)^2}{\sum_{i\in I_L}h_i+\lambda} + \frac{(\sum_{i\in I_R}g_i)^2}{\sum_{i\in I_R}h_i+\lambda} - \frac{(\sum_{i\in I}g_i)^2}{\sum_{i\in I}h_i+\lambda}]-\gamma$。对于GBDT来说，未使用二阶梯度，因此分母都是常量，即可以退化为上述lightGBM中的增益计算公式。

对于特征$j$，决策树选择最优点$d_j^*=\arg \max_d V_j(d)$计算最大的信息增益$V_j(d_j^*)$

**GOSS**

在GOSS中，首先按照梯度绝对值降序排序，选出前$a\times 100\%$个样本构成集合$A$；在剩余样本集合$A_c$中选取$b\times 100\%$个样本得到集合$B$。

因此，我们采取GOSS计算样本集合$A\cup B$的信息增益为：

$$\tilde{V}_{j}(d)=\frac{1}{n}(\frac{(\sum_{x_i\in A_l}g_i + \frac{1-a}{b}\sum_{x_i\in B_l} g_i)^2}{n_{l}^j(d)} + \frac{(\sum_{\{x_i\in A_r}g_i + \frac{1-a}{b}\sum_{x_i\in B_r} g_i)^2}{n_{r}^j(d)})$$


使用上面这种方式，我们可以得到信息增益的近似值，损失的精度非常小；同时极大地降低了计算时间复杂度。